# `PYQG` : Generating Subgrid Forcing Datasets

## 1 - Introduction & Setup
In this notebook, we'll document and showcase datasets for learning subgrid forcing parameterizations, generated with [`pyqg`](https://pyqg.readthedocs.io/en/latest/).

In [ ]:
#---------------------
#     Librairies
#---------------------
# - Xarray          : Open-source Python package for working with labeled multi-dimensional datasets.
# - Filesystem Spec : Project to provide a unified pythonic interface to local, remote and embedded file systems and bytes storage.
# - PYQG            : is a python solver for quasigeostrophic systems.     
# - JSON            : JSON is an open standard file format and data interchange format that uses human-readable text to store and transmit 
#                     data objects consisting of attribute–value pairs and arrays. It is a common data format with diverse uses in electronic 
#                     data interchange, including that of web applications with servers.

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import json
import fsspec
import xarray as xr
import pyqg
import pyqg.diagnostic_tools

# You can use the magic function %matplotlib inline to enable the inline plotting, 
# where the plots/graphs will be displayed just below the cell where your plotting commands are written. 
%matplotlib inline
plt.rcParams.update({'font.size': 13})

### What is **Globus** ?
With Globus, subscribers can move, share, & discover data via a single interface – whether your files live on a supercomputer, lab cluster, tape archive, public cloud or your laptop, you can manage this data from anywhere, using your existing identities, via just a web browser.​

https://www.globus.org/what-we-do

In [ ]:
# Datasets are hosted on globus as zarr files
def get_dataset(path, base_url = "https://g-402b74.00888.8540.data.globus.org"):
    mapper = fsspec.get_mapper(f"{base_url}/{path}.zarr")
    return xr.open_zarr(mapper, consolidated=True)

### What is the **organization** of the dataset ?
- Each dataset represents a `pyqg.QGModel` simulation run for 10 years
- Snapshot (= the state of a system at a particular point in time) every 1000 hours (= 40 days)
- Daily time-averaged diagnostic variables are also stored (averages start being taken at 5 years).
- The datasets are organized into the following [`zarr`](https://zarr.readthedocs.io/en/stable/) files:

```
eddy/
    high_res.zarr
    low_res.zarr
    forcing1.zarr
    forcing2.zarr
    forcing3.zarr
jet/
    high_res.zarr
    low_res.zarr
    forcing1.zarr
    forcing2.zarr
    forcing3.zarr
```

## 2 - `PYQG` configurations

Before digging into the variables and the different types of datasets we present, let's describe the the top-level directories (`eddy` and `jet`). These directories represent different configurations in which we run `pyqg.QGModel` simulations. These configurations vary parameters like:
- `beta`  (slope of Coriolis parameter)
- `delta` (ratio of layer widths)
- `rek`   (strength of bottom friction). 

Each leads to qualitatively different behavior, as can be seen below:

In [ ]:
eddy_high_res = get_dataset('eddy/high_res').isel(run=0)
jet_high_res  = get_dataset('jet/high_res').isel(run=0)

In [ ]:
# If you want, you can also load the low resolution datasets (to make plots later on)
eddy_low_res = get_dataset('eddy/low_res').isel(run=0)
jet_low_res  = get_dataset('jet/low_res').isel(run=0)

In [ ]:
#-----------
# Parameters
#-----------
# Index of the time slice you want to plot the results from
t = -2 

# Level in the model where you want to observe (Thera 2 of them)
level = 0

#---------
# Plotting
#---------
plt_name = f"Timestep = {t} - Level = {level} - PV Snapshots"
plt.figure(figsize=(12,5)).suptitle(plt_name)
plt.subplot(121); eddy_high_res.q.isel(lev = level, time = t).plot(); plt.title("Eddy config")
plt.subplot(122);  jet_high_res.q.isel(lev = level, time = t).plot(); plt.title("Jet config")
plt.tight_layout()

Each dataset also stores the parameters passed to `pyqg.QGModel` which were used to generate it:

In [ ]:
# Loading the parameters in JSON format (human readable)
param_eddy = json.loads(eddy_high_res.attrs['pyqg_params'])
param_jets = json.loads(jet_high_res.attrs['pyqg_params'])

print("Eddy Configuration Parameters :\n")
print(type(json.loads(eddy_high_res.attrs['pyqg_params'])))
print("\nJets Configuration Parameters :\n")
print(param_jets)

In [ ]:
# If you want to run new simulation, you can by simply adding a .run() !
new_eddy_model = pyqg.QGModel(**json.loads(eddy_high_res.attrs['pyqg_params'])) #.run()
new_jet_model  = pyqg.QGModel(**json.loads(jet_high_res.attrs['pyqg_params']))  #.run()

## 3 - `PYQG` Types of datasets

Alright, so now let's discuss the structure within each configuration-specific subdirectory:

- `high_res.zarr` contains snapshots and diagnostics for **high resolution** eddy-resolving models (`nx=256`), where the grid length is about 4x the deformation radius.


- `low_res.zarr` contains snapshots and diagnostics for **low resolution** eddy-permitting models (`nx=64`), where the grid length is slightly less than the deformation radius.


- `forcing{1,2,3}.zarr` contain high resolution snapshots that have been **_filtered and coarsened_ down** to low resolution, along with associated _subgrid forcing_ variables. There are three of these datasets because we experiment with three different operators for filtering and coarse-graining the high-resolution potential vorticity:
    * `forcing1.zarr` uses spectral truncation followed by [the sharp spectral filter used internally by `pyqg`](https://pyqg.readthedocs.io/en/latest/equations/notation_twolayer_model.html#marching-forward)


    * `forcing2.zarr` uses spectral truncation followed by a Gaussian spectral filter, following e.g. [Guan et al. 2021](https://doi.org/10.1016/j.jcp.2022.111090)

    
    * `forcing3.zarr` uses [GCM-Filters](https://gcm-filters.readthedocs.io/en/latest/) to filter the PV in real space, followed by averaging and coarsening in real space.
    
    
To **learn more** about these **filtering techniques**, go check [this notebook](./subgrid_forcing.ipynb).

In [ ]:
#--------------------
# Eddy Configuration
#--------------------
# Note:
# In order to load a SMALL subset of the datasets, you can use the function .load() !
eddy_forcing1 = get_dataset('eddy/forcing1').isel(run=0).load()
eddy_forcing2 = get_dataset('eddy/forcing2').isel(run=0).load()
eddy_forcing3 = get_dataset('eddy/forcing3').isel(run=0).load()

In [ ]:
#-------------------
# Jet Configuration
#-------------------
jet_forcing1 = get_dataset('jet/forcing1').isel(run=0).load()
jet_forcing2 = get_dataset('jet/forcing2').isel(run=0).load()
jet_forcing3 = get_dataset('jet/forcing3').isel(run=0).load()

## 4 - `PYQG` Variables stored

### **State** variables 

Each dataset stores a large number of variables. These include the following state variables:

- `q`: Potential vorcitity
- `u`: $x$-velocity relative to the background flow
- `v`: $y$-velocity relative to the background flow
- `ufull` and `vfull` - corresponding variables but including the background flow
- `streamfunction`

### **Forcing** variables

For the forcing datasets (`forcing{1,2,3}.zarr`), we include the following additional variables describing discrepancies between the high-resolution model and its coarsened counterpart (where $\overline{(\,)}$ denotes the coarse-graining operator):

- `q_subgrid_forcing`: $S_q \equiv \overline{(\mathbf{u} \cdot \nabla)q} - (\overline{\mathbf{u}} \cdot \overline{\nabla})\overline{q}$
- `u_subgrid_forcing`: $S_u \equiv \overline{(\mathbf{u} \cdot \nabla)u} - (\overline{\mathbf{u}} \cdot \overline{\nabla})\overline{u}$
- `v_subgrid_forcing`: $S_v \equiv \overline{(\mathbf{u} \cdot \nabla)v} - (\overline{\mathbf{u}} \cdot \overline{\nabla})\overline{v}$
- `uq_subgrid_flux`: $\phi_{uq} \equiv \overline{uq} - \bar{u}\bar{q}$
- `vq_subgrid_flux`: $\phi_{vq} \equiv \overline{vq} - \bar{v}\bar{q}$. Note that $\nabla \cdot \langle\phi_{uq}, \phi_{vq}\rangle = S_q$.
- `uu_subgrid_flux`: $\phi_{uu} \equiv \overline{u^2} - \bar{u}^2$
- `vv_subgrid_flux`: $\phi_{vv} \equiv \overline{v^2} - \bar{v}^2$
- `uv_subgrid_flux`: $\phi_{uv} \equiv \overline{uv} - \bar{u}\bar{v}$. Note that $\nabla \cdot \langle\phi_{uu}, \phi_{uv}\rangle = S_u$ and $\nabla \cdot \langle\phi_{uv}, \phi_{vv}\rangle = S_v$.
- `dqdt_bar`: PV tendency from the high-resolution model, filtered and coarsened to low resolution
- `dqbar_dt`: PV tendency from the low-resolution model, initialized at $\overline{q}$. Note that `dqdt_bar - dqbar_dt` 
   
   can be an alternative to $S_q$ (it's very similar, except it also accounts for numerical dissipation).

One use-case for these datasets is to learn data-driven parameterizations which predict certain of these forcing variables (either $S_q$, both $S_u$ and $S_v$, or an equivalent flux term) from the resolved low-resolution state variables.

### **Plotting** state variables

In [ ]:
def plotStateVariables(model, model_name, timestep = -1, level = 0):

    # Base name for naming the plot
    plt_name_base = f"{model_name} - Timestep = {timestep} - Level = {level} -"

    # Potential vorcitity
    plt.figure(figsize=(12,9)).suptitle(f"{plt_name_base} Potential vorcitity")
    model.q.isel(lev = level, time = timestep).plot(cbar_kwargs = dict(label = "Upper PV Forcing [$s^{-2}$]"))
    plt.savefig(f"../graphs/dataset/State_Variables/Potential_Vorticity/{plt_name_base} Potential vorcitity.png")

    # U
    plt.figure(figsize=(12,9)).suptitle(f"{plt_name_base} U")
    model.u.isel(lev = level, time = timestep).plot(cbar_kwargs = dict(label = "UNKNOWN"))
    plt.savefig(f"../graphs/dataset/State_Variables/Speed_u/{plt_name_base} U.png")

    # V 
    plt.figure(figsize=(12,9)).suptitle(f"{plt_name_base} V")
    model.v.isel(lev = level, time = timestep).plot(cbar_kwargs = dict(label = "UNKNOWN"))
    plt.savefig(f"../graphs/dataset/State_Variables/Speed_v/{plt_name_base} V.png")

    # U-Full
    plt.figure(figsize=(12,9)).suptitle(f"{plt_name_base} U-Full")
    model.ufull.isel(lev = level, time = timestep).plot(cbar_kwargs = dict(label = "UNKNOWN"))
    plt.savefig(f"../graphs/dataset/State_Variables/Speed_u_full/{plt_name_base} U-Full.png")

    # V-Full
    plt.figure(figsize=(12,9)).suptitle(f"{plt_name_base} V-Full")
    model.vfull.isel(lev = level, time = timestep).plot(cbar_kwargs = dict(label = "UNKNOWN"))
    plt.savefig(f"../graphs/dataset/State_Variables/Speed_v_full/{plt_name_base} V-Full.png")

    # Stream function
    plt.figure(figsize=(12,9)).suptitle(f"{plt_name_base} Stream function")
    model.streamfunction.isel(lev = level, time = timestep).plot(cbar_kwargs = dict(label = "UNKNOWN"))
    plt.savefig(f"../graphs/dataset/State_Variables/streamfunction/{plt_name_base} Stream function.png")

def plotStateVariablesGIF(model, model_name, timestep = -1, level = 0):

    # Base name for naming the plot
    plt_name_base = f"{model_name} - Timestep = {timestep} - Level = {level} -"

    # Potential vorcitity
    plt.figure(figsize=(12,9)).suptitle(f"{plt_name_base} Potential vorcitity")
    model.q.isel(lev = level, time = timestep).plot(cbar_kwargs = dict(label = "Upper PV Forcing [$s^{-2}$]"))
    plt.savefig(f"../graphs/dataset/State_Variables/Potential_Vorticity/{plt_name_base} Potential vorcitity.png")

    # Stream function
    plt.figure(figsize=(12,9)).suptitle(f"{plt_name_base} Stream function")
    model.streamfunction.isel(lev = level, time = timestep).plot(cbar_kwargs = dict(label = "UNKNOWN"))
    plt.savefig(f"../graphs/dataset/State_Variables/streamfunction/{plt_name_base} Stream function.png")

In [ ]:
#-----------
# Parameters
#-----------
# Indexes of the time slice you want to plot the results from
t = [i for i in range(25)]

# Level in the model where you want to observe (Thera 2 of them)
level = 0

for time in t:
    plotStateVariablesGIF(eddy_high_res, "Eddy - High Resolution", timestep = -time, level = level)
    plotStateVariablesGIF(jet_high_res , "Jet - High Resolution" , timestep = -time, level = level)


In [ ]:
#-----------
# Parameters
#-----------
# Index of the time slice you want to plot the results from
t = -1 

# Level in the model where you want to observe (Thera 2 of them)
level = 1

plotStateVariables(eddy_low_res , "Eddy - Low Resolution" , timestep = t, level = level)
plotStateVariables(eddy_high_res, "Eddy - High Resolution", timestep = t, level = level)
plotStateVariables(jet_low_res  , "Jet - Low Resolution"  , timestep = t, level = level)
plotStateVariables(jet_high_res , "Jet - High Resolution" , timestep = t, level = level)
plotStateVariables(eddy_forcing1, "Eddy - Forcing 1"      , timestep = t, level = level)
plotStateVariables(eddy_forcing2, "Eddy - Forcing 2"      , timestep = t, level = level)
plotStateVariables(eddy_forcing3, "Eddy - Forcing 3"      , timestep = t, level = level)
plotStateVariables(jet_forcing1,  "Jet - Forcing 1"       , timestep = t, level = level)
plotStateVariables(jet_forcing2,  "Jet - Forcing 2"       , timestep = t, level = level)
plotStateVariables(jet_forcing3, " Jet - Forcing 3"       , timestep = t, level = level)

# Note: Don't forget to clean the cell !

### **Plotting** forcing variables

In [ ]:
def plotForcingVariables(model, model_name, timestep = -1, level = 0):

    # Base name for naming the plot
    plt_name_base = f"{model_name} - Timestep = {timestep} - Level = {level} -"

    # q_subgrid_forcing
    plt.figure(figsize=(12,9)).suptitle(f"{plt_name_base} q_subgrid_forcing")
    model.q_subgrid_forcing.isel(lev = level, time = timestep).plot(cbar_kwargs = dict(label = "Upper PV Forcing [$s^{-2}$]"))
    plt.savefig(f"../graphs/dataset/Forcing_Variables/Potential_Vorticity_SF/{plt_name_base} q_subgrid_forcing.png")

    # u_subgrid_forcing
    plt.figure(figsize=(12,9)).suptitle(f"{plt_name_base} u_subgrid_forcing")
    model.u_subgrid_forcing.isel(lev = level, time = timestep).plot()
    plt.savefig(f"../graphs/dataset/Forcing_Variables/Speed_u_SF/{plt_name_base} u_subgrid_forcing.png")

    # v_subgrid_forcing
    plt.figure(figsize=(12,9)).suptitle(f"{plt_name_base} v_subgrid_forcing")
    model.v_subgrid_forcing.isel(lev = level, time = timestep).plot()
    plt.savefig(f"../graphs/dataset/Forcing_Variables/Speed_v_SF/{plt_name_base} v_subgrid_forcing.png")

    # uq_subgrid_flux
    plt.figure(figsize=(12,9)).suptitle(f"{plt_name_base} uq_subgrid_flux")
    model.uq_subgrid_flux.isel(lev = level, time = timestep).plot()
    plt.savefig(f"../graphs/dataset/Forcing_Variables/uq_SFlux/{plt_name_base} uq_subgrid_flux.png")

    # vq_subgrid_flux
    plt.figure(figsize=(12,9)).suptitle(f"{plt_name_base} vq_subgrid_flux")
    model.vq_subgrid_flux.isel(lev = level, time = timestep).plot()
    plt.savefig(f"../graphs/dataset/Forcing_Variables/vq_SFlux/{plt_name_base} vq_subgrid_flux.png")

    # uu_subgrid_flux
    plt.figure(figsize=(12,9)).suptitle(f"{plt_name_base} uu_subgrid_flux")
    model.uu_subgrid_flux.isel(lev = level, time = timestep).plot()
    plt.savefig(f"../graphs/dataset/Forcing_Variables/uu_SFlux/{plt_name_base} uu_subgrid_flux.png")

    # vv_subgrid_flux
    plt.figure(figsize=(12,9)).suptitle(f"{plt_name_base} vv_subgrid_flux")
    model.vv_subgrid_flux.isel(lev = level, time = timestep).plot()
    plt.savefig(f"../graphs/dataset/Forcing_Variables/vv_SFlux/{plt_name_base} vv_subgrid_flux.png")

    # uv_subgrid_flux
    plt.figure(figsize=(12,9)).suptitle(f"{plt_name_base} uv_subgrid_flux")
    model.uv_subgrid_flux.isel(lev = level, time = timestep).plot()
    plt.savefig(f"../graphs/dataset/Forcing_Variables/uv_SFlux/{plt_name_base} uv_subgrid_flux.png")

    # dqdt_bar
    plt.figure(figsize=(12,9)).suptitle(f"{plt_name_base} dqdt_bar")
    model.dqdt_bar.isel(lev = level, time = timestep).plot()
    plt.savefig(f"../graphs/dataset/Forcing_Variables/dqdt_bar/{plt_name_base} dqdt_bar.png")

    # dqbar_dt
    plt.figure(figsize=(12,9)).suptitle(f"{plt_name_base} dqbar_dt")
    model.dqbar_dt.isel(lev = level, time = timestep).plot()
    plt.savefig(f"../graphs/dataset/Forcing_Variables/dqbar_dt/{plt_name_base} dqbar_dt.png")


In [ ]:
#-----------
# Parameters
#-----------
# Index of the time slice you want to plot the results from
t = -1 

# Level in the model where you want to observe (Thera 2 of them)
level = 0

plotForcingVariables(eddy_forcing1, "Eddy - Forcing 1", timestep = t, level = level)
plotForcingVariables(eddy_forcing2, "Eddy - Forcing 2", timestep = t, level = level)
plotForcingVariables(eddy_forcing3, "Eddy - Forcing 3", timestep = t, level = level)
plotForcingVariables(jet_forcing1,  "Jet - Forcing 1" , timestep = t, level = level)
plotForcingVariables(jet_forcing2,  "Jet - Forcing 2" , timestep = t, level = level)
plotForcingVariables(jet_forcing3, " Jet - Forcing 3" , timestep = t, level = level)

### Diagnostic variables

We include a number of diagnostic variables to help measure how quantities like energy and enstrophy are distributed and transferred across scales. These make most sense to plot for the high- and low-resolution reference simulations (or for parameterized simulations), rather than for the forcing datasets (though they are still included).

**Power spectra:**
- `KEspec`: how much kinetic energy is stored at each $x$ and $y$ lengthscale
- `Ensspec`: how much enstrophy is stored at each $x$ and $y$ lengthscale

**Energy budget (should sum to 0):**
- `KEflux`: how kinetic energy is being transferred across lengthscales
- `APEflux`: how available potential energy is being transferred across lengthscales
- `APEgenspec`: how much new available potential energy is being generated at each scale
- `KEfrictionspec`: how much energy is being lost to bottom drag at each lengthscale
- `Dissspec`: how much energy is being lost due to numerical dissipation at each lengthscale

**Enstrophy budget (should sum to 0):**
- `ENSflux`: how enstrophy is being transferred across lengthscales
- `ENSgenspec`: how much new enstrophy is being generated
- `ENSfrictionspec`: how much enstrophy is lost to bottom drag
- `ENSDissspec`: how much enstrophy is lost to numerical dissipation

These can be plotted, e.g. as follows:

In [ ]:
energy_budget = ['KEflux','APEflux','APEgenspec','KEfrictionspec','Dissspec']
plt.figure(figsize=(10,6))
plt.title("High-res energy budget")
plt.axhline(0,color='gray', ls='--')
for term in energy_budget:
    plt.semilogx(eddy_high_res.k, eddy_high_res[term].sum('l'), label=term) 
plt.semilogx(eddy_high_res.k, np.sum([eddy_high_res[t].sum('l') for t in energy_budget], axis=0), label='Residual')
plt.legend()
plt.ylabel("Energy transfer $[m^2 s^{-3}]$")
plt.xlabel("Zonal wavenumber $[m^{-1}]$")
plt.show()

In [ ]:
enstrophy_budget = ['ENSflux','ENSgenspec','ENSfrictionspec','ENSDissspec']
plt.figure(figsize=(10,6))
plt.title("High-res enstrophy budget")
plt.axhline(0,color='gray', ls='--')
for term in enstrophy_budget:
    plt.semilogx(eddy_high_res.k, eddy_high_res[term].sum('l'), label=term) 
plt.semilogx(eddy_high_res.k, np.sum([eddy_high_res[t].sum('l') for t in enstrophy_budget], axis=0), label='Residual')
plt.legend()
plt.ylabel("Enstrophy transfer $[s^{-3}]$")
plt.xlabel("Zonal wavenumber $[m^{-1}]$")
plt.show()

The `xarray.Dataset` objects themselves also contain some documentation for each variable:

In [ ]:
eddy_forcing1